In [1]:
import pandas as pd
import spacy
import emoji
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
from Twitterpreprocessor import TwitterPreprocessor

In [2]:
data = pd.read_csv("./Data/data+descvf.csv")


In [5]:

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
def is_bot_profile(profile_description):
    # remove emojis
    profile_description = TwitterPreprocessor(str(profile_description)).desc_preprocess().text
    print("profile_description",profile_description)
    # tokenize the text with spaCy
    doc = nlp(profile_description)
    print("doc",doc)
    # remove stop words and punctuation
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop and token.lemma_ not in stop_words]
    print(tokens)
    # count the frequency of different parts-of-speech
    pos_tags = [token.pos_ for token in doc]
    pos_counts = Counter(pos_tags)
    print("pos count",pos_counts)
    # look for patterns typical of bot accounts
    if pos_counts['PRON'] >= 3 and pos_counts['NOUN'] >= 3:
        return True
    elif pos_counts['VERB'] >= 2 and pos_counts['ADJ'] >= 2:
        return True
    else:
        return False
#  "I am a bot that tweets about sports and technology. Follow me for updates!"
profile_description ="I am a bot that tweets about sports and technology. Follow me for updates!"
is_bot = is_bot_profile(profile_description)
print(f"Profile description: {profile_description}")
print(f"Is bot? {is_bot}")


profile_description  am bot that tweets about sports and technology follow me for updates
doc  am bot that tweets about sports and technology follow me for updates
[' ', 'bot', 'tweet', 'sport', 'technology', 'follow', 'update']
pos count Counter({'NOUN': 4, 'PRON': 2, 'VERB': 2, 'ADP': 2, 'SPACE': 1, 'AUX': 1, 'CCONJ': 1})
Profile description: I am a bot that tweets about sports and technology. Follow me for updates!
Is bot? False


In [1]:
# import spacy
# import emoji
# from nltk.corpus import stopwords
# from collections import Counter

# nlp = spacy.load("en_core_web_sm")
# stop_words = set(stopwords.words('english'))

# def is_bot_profile(profile_description):
#     # remove emojis
#     # profile_description = emoji.get_emoji_regexp().sub(r'', profile_description)
#     # lowercase the text
#     profile_description = profile_description.lower()
#     # tokenize the text with spaCy
#     doc = nlp(profile_description)
#     print(doc)
#     # remove stop words and punctuation
#     tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop and token.lemma_ not in stop_words]
#     print(tokens)
#     # count the frequency of different parts-of-speech
#     pos_tags = [token.pos_ for token in doc]
#     print(pos_tags)
#     pos_counts = Counter(pos_tags)
#     # look for patterns typical of bot accounts
#     if pos_counts['PRON'] >= 3 and pos_counts['NOUN'] >= 3:
#         return True
#     elif pos_counts['VERB'] >= 2 and pos_counts['ADJ'] >= 2:
#         return True
#     else:
#         return False
# profile_description = "I am a bot that tweets about sports and technology. Follow me for updates!"
# is_bot = is_bot_profile(profile_description)
# print(f"Profile description: {profile_description}")
# print(f"Is bot? {is_bot}")

i am a bot that tweets about sports and technology. follow me for updates!


AttributeError: 'spacy.morphology.Morphology' object has no attribute 'tag_map'

In [3]:
# import nltk
# import spacy

# from nltk.tokenize import word_tokenize

# # Load the dataset
# dataset = pd.read_csv("./Data/datafromAPI.csv")

# # Clean the text data
# dataset['description'] = dataset['description'].apply(lambda x :TwitterPreprocessor(str(x)).desc_preprocess().text)
# dataset['description'] = dataset['description'].fillna('')
# # Define function to lemmatize the tokens
# nlp = spacy.load('en_core_web_sm')
# def lemmatize_tokens(tokens):
#     doc = nlp(" ".join(tokens))
#     return [token.lemma_ for token in doc]

# # Tokenize and lemmatize the text data
# dataset['description_tokens'] = dataset['description'].apply(lambda x: lemmatize_tokens(word_tokenize(x)))

# # Remove stop words
# stop_words = set(nltk.corpus.stopwords.words('english'))
# dataset['description_tokens'] = dataset['description_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
# dataset.to_csv('./Data/description_tokens.csv', index=False)
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Convert the list of tokens to a string
# dataset['description_tokens'] = dataset['description_tokens'].apply(lambda x: ' '.join(x))


C:\Users\Chams\AppData\Local\Temp\ipykernel_16676\774891700.py:7: DtypeWarning: Columns (0,3,4,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("./Data/datafromAPI.csv")


In [8]:
# # Add the dense array to the dataset
# dataset['description_tokens'] = X_array.tolist()

In [ ]:
# dataset.to_csv('./Data/description_tokensv2.csv', index=False)

start prepare desc

In [ ]:
# import pandas as pd
# import re
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer

# # Load dataset
# df = pd.read_csv('./Data/data+descvf.csv')

# # Remove irrelevant or duplicated data
# df = df.drop(['description',  
#          'account_type'], axis=1)

# # Remove special characters and punctuation, and convert text to lowercase
# df['description'] = df['description'].apply(lambda x :TwitterPreprocessor(str(x)).desc_preprocess().text)

# # Tokenize text
# df['description'] = df['description'].apply(lambda x: x.split())

# # Remove stop words
# stop_words = set(stopwords.words('english'))
# df['description'] = df['description'].apply(lambda x: [word for word in x if word not in stop_words])

# # Stemming
# stemmer = PorterStemmer()
# df['description'] = df['description'].apply(lambda x: [stemmer.stem(word) for word in x])

# # Convert back to string
# df['description'] = df['description'].apply(lambda x: ' '.join(x))


In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
# from tensorflow.keras.models import Sequential
# # Split into training and testing sets
# train_size = int(0.8 * len(df))
# train_data = df[:train_size]
# test_data = df[train_size:]

# # Tokenize text inputs
# tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
# tokenizer.fit_on_texts(train_data['description'])

# # Convert text inputs to sequences of integers
# train_sequences = tokenizer.texts_to_sequences(train_data['description'])
# test_sequences = tokenizer.texts_to_sequences(test_data['description'])

# # Pad sequences
# max_len = 100
# train_data = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
# test_data = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# # Build model
# model = Sequential([
#     Embedding(10000, 128, input_length=max_len),
#     LSTM(64),
#     Dense(64, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

# # Compile model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Fit model
# history = model.fit(train_data, train_data['label'], epochs=10, validation_data=(test_data, test_data['label']))

# # Evaluate model
# loss, accuracy = model.evaluate(test_data, test_data['label'])
# print(f'Test loss: {loss:.4f}')
# print(f'Test accuracy: {accuracy:.4f}')

word2vec

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import pandas as pd
from collections import Counter
import numpy as np
from Twitterpreprocessor import TwitterPreprocessor

# download required NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# Load dataset
df = pd.read_csv('./Data/data+descvf.csv')


# Remove special characters and punctuation, and convert text to lowercase
df['description'] = df['description'].apply(lambda x :TwitterPreprocessor(str(x)).partially_preprocess().text)
df = df.dropna(subset=['description'])
# example list of string descriptions
descriptions = df['description'] 

# create list of stopwords to remove
stop_words = set(stopwords.words('english'))

# create WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# initialize list of lemmatized tokens for each description
lemmatized_tokens_list = []

nan_value = float("NaN")


descriptions.replace("nan", nan_value, inplace=True)
descriptions=descriptions.dropna()
descriptions

1        television producer emmy award winner disney e...
2                                          self discovery 
4        productor de televisión embajador de contacto ...
6        im dmstic engineer married to hnk for yrs pups...
7        shenanigatrix wordsmith events social media co...
                               ...                        
34039    self made plus model ceo siswim world traveler...
34040                                   gazetecijournalist
34041    modelo actriz venezolana venezuelan model actr...
34042                              indianactorneed no more
34043    poeta autor escritor acadêmico dois livros pub...
Name: description, Length: 27034, dtype: object

In [22]:

# loop through each description, performing POS tagging, stopword removal, and lemmatization
from lingua import Language, LanguageDetectorBuilder
  
detector = LanguageDetectorBuilder.from_all_languages().build()
x=['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']
for description in descriptions:
    # create list of stopwords to remove
    C=detector.detect_language_of(description)
    try :
        if C.name.lower() in x:
            stop_words = set(stopwords.words(C.name.lower()))
        else :
            stop_words = set(stopwords.words("english"))
    except :
            print(description)
            stop_words = set(stopwords.words("english"))
    # perform tokenization
    tokens = word_tokenize(description.lower())

    # perform POS tagging
    pos_tags = nltk.pos_tag(tokens)

    # initialize list of lemmatized tokens for this description
    lemmatized_tokens = []

    # loop through POS tagged tokens, removing certain parts of speech and lemmatizing others
    for token in tokens:
        # if pos.startswith('N') or pos.startswith('J'):
            # keep nouns and adjectives, and lemmatize them
            lemma = lemmatizer.lemmatize(token)
            if lemma not in stop_words:
                lemmatized_tokens.append(lemma)

    # add lemmatized tokens for this description to list
    lemmatized_tokens_list.append(lemmatized_tokens)






 
 

¯¯
 
 
 





 
 

𝙄𝙣𝙨𝙩𝙖𝙜𝙧𝙖𝙢 𝙤𝙡𝙖𝙟𝙤𝙧𝙙𝙖𝙣 • 𝙎𝙣𝙖𝙥𝙘𝙝𝙖𝙩 𝙤𝙡𝙖𝙟𝙤𝙧𝙙𝙖𝙣 • 𝙈𝙚𝙙𝙞𝙖 𝙚𝙣𝙦𝙪𝙞𝙧𝙞𝙚𝙨 𝙘𝙤𝙣𝙩𝙖𝙘𝙩 
 

 
𝘊𝘩𝘢𝘮𝘱𝘪𝘰𝘯 𝘥𝘶 𝘔𝘰𝘯𝘥𝘦 𝘍𝘊 𝘉𝘢𝘳𝘤𝘦𝘭𝘰𝘯𝘢 𝘤𝘰𝘯𝘵𝘢𝘤𝘵𝘤𝘰𝘮
 

“𝑳𝒆𝒕𝒕𝒊𝒏𝒈 𝒑𝒆𝒐𝒑𝒍𝒆 𝒅𝒐𝒘𝒏 𝒊𝒔 𝒎𝒚 𝒕𝒉𝒊𝒏𝒈 𝒃𝒂𝒃𝒚 — 𝒇𝒊𝒏𝒅 𝒚𝒐𝒖𝒓𝒔𝒆𝒍𝒇 𝒏𝒆𝒘 𝒈𝒊𝒈” 


𝗕𝗲𝗛𝗮𝗽𝗽𝘆𝗜𝘁𝗗𝗿𝗶𝘃𝗲𝘀𝗣𝗲𝗼𝗽𝗹𝗲𝗖𝗿𝗮𝘇𝘆 
 
 
 



 

 


 
 
 

 

 
 
 



 
 ♡

 
☆





 

 



 


 ‐ 










ั

 






 ☥ 




◞‸◟☞ 𝙷𝚎𝚊𝚟𝚎𝚗’ 𝙶𝚊𝚝𝚎 𝚍𝚛𝚘𝚙𝚘𝚞𝚝 


 

 


 
 

 







••


 
 
 




 
 


 


 
•• 

 ◠‿◠ 
 

 
 𝔱𝔥𝔢 𝔬𝔫𝔢 𝔞𝔫𝔡 𝔬𝔫𝔩𝔶 𝔩𝔦𝔳𝔦𝔫𝔤 𝔡𝔢𝔠𝔢𝔞𝔰𝔢𝔡 



 
𝒯𝒽ℯ 𝒟𝒾𝓋𝒾𝓃ℯ ℱℯ𝓂𝒾𝓃𝒾𝓃ℯ ❥






 


 


☆。



 

𝐀𝐓𝐋𝐀𝐒 𝐀𝐥𝐢𝐬 


♡♡

 


 


𝑅𝑒𝓈𝒾𝓈𝓉𝒶𝓃𝒸𝑒 𝐿𝒾𝒷𝑒𝓇𝒶𝓁 𝒟𝑒𝓂 𝐵𝓁𝓊𝑒 𝒲𝒶𝓋𝑒 𝒮𝓃𝑜𝓌𝒻𝓁𝒶𝓀𝑒 𝒱𝑜𝓉𝑒𝐵𝓁𝓊𝑒𝒩𝑜𝑀𝒶𝓉𝓉𝑒𝓇𝒲𝒽𝑜 𝒟𝑀’

☆ 


 
 


 

 

 
 


 
 
𝕽𝖊𝖆𝖑 𝖎𝖘 𝕽𝖆𝖗𝖊 








 

†


¯¯
 






 
ｅｌｅｃｔｉｏｎ ｃｏｖｅｒａｇｅ


 
 

 

 




△⃒⃘ ⚯͛ • • 


 




 




 



！！！

 




♡ ♡

 
 

 

♡
𝐓𝐡𝐞 𝐖𝐚𝐥𝐤𝐢𝐧𝐠 𝐃𝐞𝐚𝐝 𝐂𝐀𝐑𝐘𝐋 𝐺𝑒𝑡 𝑜𝑛 𝑡ℎ𝑒 𝑏𝑖𝑘𝑒 𝑎𝑛𝑑 𝑔𝑜 




 

 

 
 
 
𝖌𝖔𝖙𝖍 𝖆𝖓𝖌𝖊𝖑 𝖘𝖎𝖓𝖓𝖊𝖗
 

 





｡･･★｡



 



 





 


 

 

 
 
𝑃𝑣𝑟𝑝𝑙𝑒𝑀𝑥𝑏𝐿𝑙𝑐 𝑬𝒔𝗍 𝟐𝟎𝟏𝟐

 




 


𝗀𝗂𝗋𝗅 𝖻𝗈𝗌𝗌 𝗌𝗍𝗈𝗋𝗒𝗅𝗂𝗇𝖾 𝗃𝖾𝗐𝖾𝗅𝗋𝗒


 





 

 



 
𝖨𝖦 𝖽𝗋𝖾𝖿𝖺𝖼𝖾 • 𝖲𝖢 𝖠𝗇𝖽𝗋𝖾𝖺 𝖣𝖺𝗏𝗂𝗌

In [24]:
import json

In [25]:
# lemmatized_tokens_list.to_json('lemmatized_tokens_list.json', orient='records')
with open('lemmatized_tokens_list.json', 'w') as f:
    json.dump(lemmatized_tokens_list, f)

In [14]:
# train Word2Vec model on all lemmatized tokens
model = Word2Vec(lemmatized_tokens_list, min_count=5)

# get vector representation of each description by averaging the vectors of its constituent lemmatized tokens
description_vectors = []
for lemmatized_tokens in lemmatized_tokens_list:
    description_vector = sum(model.wv[token] for token in lemmatized_tokens) / len(lemmatized_tokens)
    description_vectors.append(description_vector)

# print vector representation of each description
for i, vector in enumerate(description_vectors):
    print(f"Description {i+1} vector: {vector}")

KeyError: "Key 'zbb' not present"

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import pandas as pd
from collections import Counter
import numpy as np
from Twitterpreprocessor import TwitterPreprocessor
df = pd.read_csv('./Data/data+descvf.csv')


In [12]:
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

set(stopwords.words('english'))

text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""

stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(text) 
    
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 

Stem_words = []
ps =PorterStemmer()
for w in filtered_sentence:
    rootWord=ps.stem(w)
    Stem_words.append(rootWord)
print(filtered_sentence)
print(Stem_words)

['He', 'determined', 'drop', 'litigation', 'monastry', ',', 'relinguish', 'claims', 'wood-cuting', 'fishery', 'rihgts', '.', 'He', 'ready', 'becuase', 'rights', 'become', 'much', 'less', 'valuable', ',', 'indeed', 'vaguest', 'idea', 'wood', 'river', 'question', '.']
['he', 'determin', 'drop', 'litig', 'monastri', ',', 'relinguish', 'claim', 'wood-cut', 'fisheri', 'rihgt', '.', 'he', 'readi', 'becuas', 'right', 'becom', 'much', 'less', 'valuabl', ',', 'inde', 'vaguest', 'idea', 'wood', 'river', 'question', '.']


In [19]:
from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH]
detector = LanguageDetectorBuilder.from_all_languages().build()
C=detector.detect_language_of("teste teste")
C.name

'ROMANIAN'

In [9]:

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string


In [10]:

import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
